In [ ]:
import logging
import ipywidgets as widgets
from beamlime.kafka.sink import KafkaSink, serialize_dataarray_to_f144
from beamlime import Message, StreamId, StreamKind

instrument = 'bifrost'

# Create output widget for displaying messages
output_widget = widgets.Output(layout={'border': '1px solid black', 'height': '500px'})


# Custom logging handler that writes to the output widget
class WidgetHandler(logging.Handler):
    def __init__(self, output_widget):
        super().__init__()
        self.output_widget = output_widget

    def emit(self, record):
        msg = self.format(record)
        with self.output_widget:
            print(msg)


# Create logger that writes to the output widget
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# Clear any existing handlers
logger.handlers.clear()
widget_handler = WidgetHandler(output_widget)
widget_handler.setFormatter(
    logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
)
logger.addHandler(widget_handler)

sink = KafkaSink(
    kafka_config={"bootstrap.servers": "localhost:9092"},
    instrument=instrument,
    serializer=serialize_dataarray_to_f144,
    logger=logger,
)

In [ ]:
import time
import scipp as sc


def publish_now(value: float, stream_name: str):
    """Helper function to publish a message immediately"""
    da = sc.DataArray(
        sc.scalar(value), coords={'time': sc.scalar(time.time_ns(), unit='ns')}
    )
    msg = Message(value=da, stream=StreamId(kind=StreamKind.LOG, name=stream_name))
    sink.publish_messages([msg])
    logger.info("Stream '%s' - Published message with value: %s", stream_name, value)

In [ ]:
import ipywidgets as widgets
from IPython.display import display


def create_stream_slider(
    stream_name, description, initial_value=0.0, min_val=0.0, max_val=10.0, step=0.1
):
    """
    Create a slider widget that publishes messages to a specified stream.

    Args:
        stream_name: Name of the stream to publish to
        description: Description text for the slider
        initial_value: Initial slider value
        min_val: Minimum slider value
        max_val: Maximum slider value
        step: Step size for slider

    Returns:
        FloatSlider widget configured with callback
    """

    def on_slider_change(change):
        """Callback function that publishes a message when slider value changes"""
        value = change['new']
        publish_now(value, stream_name)
        logger.info(
            "Stream '%s' - Published message with value: %s", stream_name, value
        )

    slider = widgets.FloatSlider(
        value=initial_value,
        min=min_val,
        max=max_val,
        step=step,
        description=description,
        continuous_update=True,
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'},
    )

    slider.observe(on_slider_change, names='value')
    return slider


In [ ]:
detector_angle_slider = create_stream_slider(
    stream_name='detector_angle',
    description='Detector Angle:',
    initial_value=1.2,
    min_val=0.0,
    max_val=10.0,
    step=0.1,
)

sample_temperature_slider = create_stream_slider(
    stream_name='sample_temperature',
    description='Sample Temp (K):',
    initial_value=300.0,
    min_val=0.0,
    max_val=1000.0,
    step=1.0,
)

magnetic_field_slider = create_stream_slider(
    stream_name='magnetic_field',
    description='Magnetic Field (T):',
    initial_value=0.0,
    min_val=-5.0,
    max_val=5.0,
    step=0.01,
)

# Display all sliders and output widget
display(
    widgets.VBox(
        [
            detector_angle_slider,
            sample_temperature_slider,
            magnetic_field_slider,
            output_widget,
        ]
    )
)